In [1]:
import os

In [2]:
%pwd

'd:\\HeartAttackAnalysis\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\HeartAttackAnalysis'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path


In [6]:
from src.HeartAttackAnalysis.constants import *
from src.HeartAttackAnalysis.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,

        )
        return data_ingestion_config

In [8]:
import os 
import urllib.request as request
from src.HeartAttackAnalysis.logging.logger import logging


In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        """
        Initializes the DataIngestion class with the given configuration.
        """
        self.config = config

    def download_file(self):
        """
        Downloads the file from the specified URL in the configuration and stores it in the specified location.
        """
        # Ensure the root directory exists
        os.makedirs(self.config.root_dir, exist_ok=True)

        # Construct the complete path for the local data file
        local_file_path = os.path.join(self.config.root_dir, self.config.local_data_file)

        # Download the file if it doesn't already exist
        if not os.path.exists(local_file_path):
            try:
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=local_file_path
                )
                logging.info(f"{filename} downloaded successfully with the following info:\n{headers}")
            except Exception as e:
                logging.error(f"Error occurred while downloading the file: {e}")
                raise e
        else:
            logging.info(f"File already exists at {local_file_path}")

In [10]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:
    raise e